In [1]:
import scipy.io as sci
import os

import numpy as np
import pandas as pd

In [2]:
mat = sci.loadmat('metadata.mat') # loadmat을 하는 경우 mat파일을 python dictionary형태로 load
mat

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Sun Oct 25 12:52:39 2020',
 '__version__': '1.0',
 '__globals__': [],
 'labelRecNum': array([[   2,    2,    2, ..., 3523, 3523, 3523]], dtype=int16),
 'frameIndex': array([[   0,    1,    2, ..., 2988, 2989, 2990]], dtype=int32),
 'labelDotXCam': array([[ 1.064     ,  1.064     ,  1.064     , ..., 10.24670196,
         10.24670196, 10.24670196]]),
 'labelDotYCam': array([[-6.0055    , -6.0055    , -6.0055    , ...,  1.97545637,
          1.97545637,  1.97545637]]),
 'labelFaceGrid': array([[ 6, 10, 13, 13],
        [ 6, 10, 13, 13],
        [ 6, 10, 13, 13],
        ...,
        [ 7,  7,  7,  7],
        [ 7,  7,  7,  7],
        [ 7,  7,  7,  7]], dtype=uint8),
 'labelTrain': array([[1, 1, 1, ..., 0, 0, 0]], dtype=uint8),
 'labelVal': array([[0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'labelTest': array([[0, 0, 0, ..., 1, 1, 1]], dtype=uint8)}

In [3]:
np.set_printoptions(threshold=10, linewidth=10) #np.array의 ... 조정, np.set_printoptions(threshold=np.inf, linewidth=np.inf)

In [4]:
mat['labelRecNum']

array([[   2,
           2,
           2,
        ...,
        3523,
        3523,
        3523]],
      dtype=int16)

In [7]:
labelRecNum = pd.DataFrame(mat['labelRecNum'].T)
frameIndex = pd.DataFrame(mat['frameIndex'].T)
labelDotXCam = pd.DataFrame(mat['labelDotXCam'].T)
labelDotYCam = pd.DataFrame(mat['labelDotYCam'].T)
# labelFaceGrid = pd.DataFrame(mat['labelFaceGrid'].T)
labelTrain = pd.DataFrame(mat['labelTrain'].T)
labelVal = pd.DataFrame(mat['labelVal'].T)
labelTest = pd.DataFrame(mat['labelTest'].T)

result = pd.concat([labelRecNum, frameIndex, labelDotXCam, labelDotYCam, labelTrain, labelVal, labelTest], axis=1)
result.head()

,0,0,0,0,0,0,0
0,2,0,1.064000,-6.005500,1,0,0
1,2,1,1.064000,-6.005500,1,0,0
2,2,2,1.064000,-6.005500,1,0,0
3,2,3,1.064000,-6.005500,1,0,0
4,2,4,3.251485,-1.993073,1,0,0


In [6]:
'''
- dataset_origin에는 동영상의 원본프레임이 보존되어 있고 dataset에는 얼굴/눈으로 인식되지 않은 frame의 사진은 삭제되어 있다.
- dataset_origin에서 얼굴 확인이 되는 frame 추출 후 그 frame에 대한 Left eye, Right eye가 추출되어 있다. (즉 얼굴/눈의 #frame은 같다.)
- dotInfo.json에 있는 Xpts, Ypts에 대한 정보는 코드에 사용되지 않음
- 총 사진의 갯수 1490959*3개 (왜냐하면 폴더가 3개이기 때문)



* metadata.mat의 정보는 폴더 하나만 고려한 기준이다 (1490959개) *


- labelRecNum : 사진의 갯수만큼 폴더명 인덱스를 부여해놓은 리스트.
                ex> 2, 2, 2 ..., 3, 3, ..., .. 
                    / 2번 폴더에 56개 사진이 있으므로 56개의 2가 존재, 3번 폴더에 86개 사진있으므로 86개만큼 3이 존재 ...
                
                
- frameIndex : 전처리된 dataset의 폴더 안에 있는 frame 번호 
                ex> 0, 1, 2 ... 97, 0, 1, 2, .... 
                    / 2번 폴더안에 97번 까지 프레임번호의 사진이 있고 그다음 3번 폴더의 프레임번호가 시작됨. 

labelDotXCam / labelDotYCam : 정확하게는 잘 모르겠는데 카메라의 중심으로부터 상대적 거리로 보임. 실험자들이 각각 다른 디바이스로 데이터를 만들어서
                                일관적인 값을 가져오기 위해 unified prediction space를 이용. 거리는 센티미터 기준이며 실제로 실험자들은
                                디바이스를 움직이면서 데이터를 만들지만 labelDotX/YCam은 카메라가 고정된 한 포지션이라 가정.
                                Ycam의 경우 핸드폰 스크린위에 카메라가 있기 때문에 negative한 값을 가질 것이며 만약 반대로 디바이스를 든 경우
                                positive한 값을 가질수도 있음. (screen.json 파일에 디바이스 그립에 따른 orientation에 대한 정보가 있다.)

labelFaceGrid : 전처리전 사진에 대한 grid의 scale은 25by25로 지정되어 있다.
                ex> 6, 10, 13, 13인 경우, 좌측 상단 위치 = 6,10 / 가로 : 13 / 세로 : 13의 face grid를 갖는다.
                
labelTrain / labelVal / labelTest : 전체 데이터 사진에 대한 Train/Val/Test Index. 
                                    랜덤샘플링을 해서 1271명은 학습데이터, 50명은 검증데이터, 150명은 테스트 데이터로 사용.
'''

'\n- dataset_origin에는 동영상의 원본프레임이 보존되어 있고 dataset에는 얼굴/눈으로 인식되지 않은 frame의 사진은 삭제되어 있다.\n- dataset_origin에서 얼굴 확인이 되는 frame 추출 후 그 frame에 대한 Left eye, Right eye가 추출되어 있다. (즉 얼굴/눈의 #frame은 같다.)\n- dotInfo.json에 있는 Xpts, Ypts에 대한 정보는 코드에 사용되지 않음\n- 총 사진의 갯수 1490959*3개 (왜냐하면 폴더가 3개이기 때문)\n\n\n\n* metadata.mat의 정보는 폴더 하나만 고려한 기준이다 (1490959개) *\n\n\n- labelRecNum : 사진의 갯수만큼 폴더명 인덱스를 부여해놓은 리스트.\n                ex> 2, 2, 2 ..., 3, 3, ..., .. \n                    / 2번 폴더에 56개 사진이 있으므로 56개의 2가 존재, 3번 폴더에 86개 사진있으므로 86개만큼 3이 존재 ...\n                \n                \n- frameIndex : 전처리된 dataset의 폴더 안에 있는 frame 번호 \n                ex> 0, 1, 2 ... 97, 0, 1, 2, .... \n                    / 2번 폴더안에 97번 까지 프레임번호의 사진이 있고 그다음 3번 폴더의 프레임번호가 시작됨. \n\nlabelDotXCam / labelDotYCam : 정확하게는 잘 모르겠는데 카메라의 중심으로부터 상대적 거리로 보임. 실험자들이 각각 다른 디바이스로 데이터를 만들어서\n                                일관적인 값을 가져오기 위해 unified prediction space를 이용. 거리는 센티미터 기준이며 실제로 실험자들은\n                                디바이스를 움직이면서 데이터를 만들지만 l

In [17]:
mat_2 = sci.loadmat('mean_face_224.mat')
mat_2

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Fri Apr 29 04:29:44 2016',
 '__version__': '1.0',
 '__globals__': [],
 'image_mean': array([[[112.809074,
          129.45522 ,
          144.93938 ],
         [111.890465,
          128.60057 ,
          144.06122 ],
         [111.20556 ,
          127.57935 ,
          143.36815 ],
         ...,
         [108.1978  ,
          124.373665,
          141.28052 ],
         [109.124214,
          125.0978  ,
          142.07185 ],
         [109.79371 ,
          125.89104 ,
          142.74463 ]],
 
        [[112.47665 ,
          129.04677 ,
          144.58781 ],
         [111.5318  ,
          128.22336 ,
          143.7069  ],
         [110.81582 ,
          127.210175,
          142.99075 ],
         ...,
         [107.88884 ,
          124.02679 ,
          140.97693 ],
         [108.785065,
          124.7553  ,
          141.75473 ],
         [109.4146  ,
          125.57967 ,
          142.38727 ]],
 
        [

In [ ]:
'''
mean_face_224 / ... /...

이건 이미지를 모두 더해서 평균을 구할 수 없으니까 각 이미지 한장에 대한 평균을 구해서 (한 차원에 3개 데이터가 있으니 RGB에 대한 평균을 빼는듯?)
빼가지고 standardization을 함 (데이터의 평균을 0으로 만듬)
'''